In [ ]:
from utils import *
np.random.seed(42)
emotions = ['happy', 'surprise', 'sad', 'angry', 'disgust', 'fear', 'neutral']
predictions = {}

In [ ]:
datasets = ['fer2013', 'rafdb']
#datasets = ['fer2013']
#datasets = ['rafdb']

#modelp = 'resemotenet'
modelp = 'vit'

In [ ]:
MAX = 150
for dataset in datasets:
    print('ds:', dataset)
    classes = []
    preds = []
    shaps = []
    weights = os.path.join(os.getcwd(), 'datasets', dataset+'_model_'+modelp+'.pth')
    for label in emotions:
        print('label:', label)
        files = np.array( list(sorted(os.listdir(os.path.join(os.getcwd(), 'datasets', dataset, 'test', label)), reverse=True) ))
        idxs = np.arange(len(files))
        idxs = shuffle(idxs, random_state=42) 
        files = files[idxs]
        c = 0
        for file_ in files:
            img = cv2.imread(os.path.join(os.getcwd(), 'datasets', dataset, 'test', label, file_))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img) 

            pred, model = detect_emotion(img, weights, modelp, emotions, True)                
            classes.append(emotions.index(label))
            preds.append(pred)
            del pred

            if c<MAX:
                preds_aux = {}
                shap_values, probs, imgs = generate_shap_images(img, model, modelp, device, emotions)
                for key in imgs:
                    pred = detect_emotion(imgs[key], weights, modelp, emotions) 
                    preds_aux[key] = pred
                    del pred
                if (c%10)==0:
                    shaps.append([label, shap_values.values[0].astype(np.float16), probs, img, imgs, preds_aux])
                else:
                    shaps.append([label, shap_values.values[0].astype(np.float16), probs, None, None, preds_aux])
                del preds_aux 
            c+=1

    with open(os.path.join(os.getcwd(), 'out', modelp+'_shaps_'+dataset+'.pkl'), 'wb') as a:
        pickle.dump(shaps, a, protocol=2)
    with open(os.path.join(os.getcwd(), 'out', modelp+'_results_'+dataset+'.pkl'), 'wb') as a:
        pickle.dump([classes, preds], a, protocol=2)